In [10]:
# Set the folder paths so that it can be used later
silver_folder_path = 'abfss://ws-container@bogomarketingdl.dfs.core.windows.net/silver'
gold_folder_path = 'abfss://ws-container@bogomarketingdl.dfs.core.windows.net/gold'

StatementMeta(rewardprogram, 26, 2, Finished, Available, Finished)

In [11]:
from pyspark.sql import functions as F

StatementMeta(rewardprogram, 26, 3, Finished, Available, Finished)

In [12]:
def ingest_table(table_name):
    df = spark.read.parquet(f"{silver_folder_path}/{table_name}")
    # Create the 'day' column by dividing 'time' by 24
    df = df.withColumn('day', ((F.col('time') / 24)+1).cast('int'))
    df = df.dropDuplicates()
    df.write.mode("overwrite").partitionBy("day","time")\
    .format("parquet").saveAsTable(f"gold_reward_program_spark.{table_name}")

StatementMeta(rewardprogram, 26, 4, Finished, Available, Finished)

In [13]:
table_names = ['transaction','offer_received','offer_viewed','offer_completed']

for table_name in table_names:
    ingest_table(table_name)
    print(f"success create table {table_name}")

StatementMeta(rewardprogram, 26, 5, Finished, Available, Finished)

success create table transaction
success create table offer_received
success create table offer_viewed
success create table offer_completed
